<a href="https://colab.research.google.com/github/Paradorn657/CNNs/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install attacut
import tensorflow as tf 
from tensorflow import keras
import random
import json
import numpy as np
from attacut import tokenize
import pickle

import re


with open("/content/sample_data/intents.json",encoding='utf-8') as file:
    data = json.load(file)

In [3]:
import gdown
gdown.download('https://drive.google.com/uc?id=0B0ZXk88koS2KV1FJN0xRX1FxaFE&export=download', 'th.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=0B0ZXk88koS2KV1FJN0xRX1FxaFE&export=download
To: /content/th.zip
122MB [00:02, 49.0MB/s]


'th.zip'

In [4]:
!unzip th.zip

Archive:  th.zip
  inflating: th.bin                  
  inflating: th.tsv                  


In [5]:
with open('th.tsv',encoding='utf-8') as f:
  vocab = []
  W = None
  vec = ''
  for line in f.readlines():
    temp = line.split('\t')
    if len(temp) == 3:
      vocab.append(temp[1])
      if int(temp[0]) % 1000 == 0:
        print('\r' + temp[0], end='')
      wordvector = np.fromstring(re.sub('\s+', ' ', vec)[1:-1], sep=' ', dtype=np.float32)
      if len(wordvector) > 0:
        if W is None:
          W = wordvector.copy()
        else:
          W = np.vstack((W, wordvector))
      vec = temp[-1]
    elif len(temp) == 1:
      vec += temp[-1]
  W = np.vstack((W, wordvector))

0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  # This is added back by InteractiveShellApp.init_path()


30000

In [6]:
len(vocab)

30225

In [7]:
def sent2idx(x):
  xidx = []
  for w in tokenize(x):
    if w in vocab:
      xidx.append(vocab.index(w))
  return xidx

In [8]:
Xtrain_idx = []
ytrain = []
labels = []
maxlen = 0
for x in data["intents"]:
  print(x["patterns"])
  for word in x["patterns"]:
    Xtrain_idx.append(sent2idx(word))
    if len(Xtrain_idx[-1]) > maxlen:
      maxlen = len(Xtrain_idx[-1])
    ytrain.append(x["tag"])
  if x["tag"] not in labels:
    labels.append(x["tag"])

['ว่าไง', 'สวัสดีครับ', 'มีใครอยู่ไหม', 'หวัดดี', 'ดีครับ']
['บาย', 'เจอกันใหม่', 'ไปละ', 'เจอกัน']
['ขอบคุณค้าบ', 'ขอบใจมาก', 'มันช่วยเหลือเราเยอะเลย']
['เปิดร้านกี่โมง', 'กี่โมงเปิดร้าน', 'เปิดร้านตอนไหน']
['ใช้บัตรเครดิตได้ไหม', 'ใช้มาสเตอร์การ์ดได้ปะ', 'ใช้เงินจ่ายได้แค่ช่องทางเดียวหรอ']


In [9]:
Xtrain_idx

[[15, 6079],
 [4740, 346],
 [7, 781, 24, 1428],
 [4188, 77],
 [77, 346],
 [4164],
 [1041, 40, 132],
 [17, 608],
 [1041, 40],
 [2261],
 [14731, 58],
 [260, 854, 458, 3609, 492],
 [205, 970, 1577, 5002],
 [1577, 5002, 205, 970],
 [205, 970, 135, 1363],
 [26, 2352, 6103, 8, 1428],
 [26, 2911, 1419, 8, 2049],
 [26, 286, 1183, 8, 908, 222, 11568]]

In [10]:
def setmaxlen(data,maxlen):
  datamaxlen = []
  for i,x in enumerate(data):
    if len(x) < maxlen:
      datamaxlen.append(np.hstack((x,np.zeros(maxlen-len(x)))))
  return datamaxlen

In [11]:
Xtrain_maxlen = setmaxlen(Xtrain_idx,maxlen)

In [12]:
Xtrain_maxlen = np.array(Xtrain_maxlen)
Xtrain_maxlen

array([[1.5000e+01, 6.0790e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [4.7400e+03, 3.4600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [7.0000e+00, 7.8100e+02, 2.4000e+01, 1.4280e+03, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [4.1880e+03, 7.7000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [7.7000e+01, 3.4600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [4.1640e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.0410e+03, 4.0000e+01, 1.3200e+02, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.7000e+01, 6.0800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [1.0410e+03, 4.0000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00],
       [2.2610e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0

In [13]:
ytrain

['ทักทาย',
 'ทักทาย',
 'ทักทาย',
 'ทักทาย',
 'ทักทาย',
 'การจากลา',
 'การจากลา',
 'การจากลา',
 'การจากลา',
 'ขอบคุณ',
 'ขอบคุณ',
 'ขอบคุณ',
 'เวลาเปิดร้าน',
 'เวลาเปิดร้าน',
 'เวลาเปิดร้าน',
 'ช่องทางจ่ายตัง',
 'ช่องทางจ่ายตัง',
 'ช่องทางจ่ายตัง']

In [14]:
ytrain_onehot = []
for i in ytrain:
  bag = [0 for _ in range(len(labels))]
  bag[labels.index(i)] = 1
  ytrain_onehot.append(bag)
ytrain_onehot = np.array(ytrain_onehot)
ytrain_onehot

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(W.shape[0], W.shape[1], name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(8, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(4)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(5,activation="softmax"))

model.compile(loss=keras.losses.mean_squared_error, optimizer=keras.optimizers.Adam())
model.get_layer('embed').set_weights([W])
model.get_layer('embed').trainable = False
model.fit(Xtrain_maxlen,ytrain_onehot[:17], epochs=500)

In [32]:
response = []
for res in data["intents"]:
  response.append(res["responses"])
response

[['สวัสดีครับผม ภราดรบอท นะ',
  'ดีใจที่ได้คุยด้วยนะครับ',
  'ว่าไงให้เราช่วยอะไร'],
 ['เจอกันใหม่น้า, ขอบคุณที่เข้ามาคุยด้วย *_* ',
  'Have a nice day เด้อ',
  'Bye! ไว้มาคุยกับเราด้วย.'],
 ['ดีใจที่ได้ช่วยนะครับ!!', 'ไม่เป็นไรค้าบยินดีช่วยเหลือ', 'โอเคครับ'],
 ['เราเปิดทุกวันเวลา 9 โมง- 3 ทุ่ม', 'Our hours are 9am-9pm every day'],
 ['We accept VISA, Mastercard and AMEX', 'We accept most major credit cards']]

In [45]:
def talkwithbot(message):
  message_idx = []
  message_idx.append(sent2idx(message))
  message_maxlen = setmaxlen(message_idx,maxlen)
  labels_predict = labels[np.argmax(model.predict(message_maxlen)[0])]
  response_list = response[labels.index(labels_predict)]
  return response_list[random.randint(0,len(response_list)-1)]

In [47]:
while True:
  message = input("you > ")
  bot_reply = talkwithbot(message)
  print("bot > {}".format(bot_reply))

you > หวัดดี
bot > ดีใจที่ได้คุยด้วยนะครับ
you > จ่ายตัง
bot > ไม่เป็นไรค้าบยินดีช่วยเหลือ
you > บัตรเครดิตได้ไหม
bot > We accept VISA, Mastercard and AMEX
you > กี่โมงละ
bot > เจอกันใหม่น้า, ขอบคุณที่เข้ามาคุยด้วย *_* 
you > เออ
bot > Bye! ไว้มาคุยกับเราด้วย.
you > ไปละนะครับ
bot > Bye! ไว้มาคุยกับเราด้วย.
you > หวัดดีค้าบ
bot > ดีใจที่ได้คุยด้วยนะครับ
you > ว่า
bot > สวัสดีครับผม ภราดรบอท นะ
you > ตอนไหน
bot > Our hours are 9am-9pm every day


KeyboardInterrupt: ignored